# Clasificación Voting Classifier

## Importación de librerías

In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

## Carga de datos

In [3]:
data_path = './data/'

train_data = pd.read_csv(f'{data_path}train_data.csv')
test_data = pd.read_csv(f'{data_path}test_data.csv')

df_reduce_mrmr = pd.read_csv(f'{data_path}X_train_reduce_mrmr.csv')
df_reduce_mrmr_instances = pd.read_csv(f'{data_path}df_reduce_mrmr_instances.csv')
df_reduce_mrmr_instances_hard = pd.read_csv(f'{data_path}df_reduce_mrmr_instances_hard.csv')
df_reduce_mrmr_instances_GLVQ = pd.read_csv(f'{data_path}df_reduce_mrmr_instances_GLVQ.csv')

df_X_train_reduce_RFC = pd.read_csv(f'{data_path}df_X_train_reduce_RFC.csv')
df_reduce_RFC_instances = pd.read_csv(f'{data_path}df_reduce_RFC_instances.csv')
df_reduce_RFC_instances_hard = pd.read_csv(f'{data_path}df_reduce_RFC_instances_hard.csv')
df_reduce_RFC_instances_GLVQ = pd.read_csv(f'{data_path}df_reduce_RFC_instances_GLVQ.csv')

print("Datos cargados exitosamente:")
print(f"train_data: {train_data.shape}")
print(f"df_reduce_mrmr: {df_reduce_mrmr.shape}")
print(f"df_reduce_mrmr_instances: {df_reduce_mrmr_instances.shape}")
print(f"df_reduce_mrmr_instances hard: {df_reduce_mrmr_instances_hard.shape}")
print(f"df_reduce_mrmr_instances_GLVQ: {df_reduce_mrmr_instances_GLVQ.shape}")
print(f"df_X_train_reduce_RFC: {df_X_train_reduce_RFC.shape}")
print(f"df_reduce_RFC_instances: {df_reduce_RFC_instances.shape}")
print(f"df_reduce_RFC_instances hard: {df_reduce_RFC_instances_hard.shape}")
print(f"df_reduce_RFC_instances_GLVQ: {df_reduce_RFC_instances_GLVQ.shape}")

Datos cargados exitosamente:
train_data: (256326, 31)
df_reduce_mrmr: (256326, 11)
df_reduce_mrmr_instances: (886, 11)
df_reduce_mrmr_instances hard: (886, 11)
df_reduce_mrmr_instances_GLVQ: (2, 11)
df_X_train_reduce_RFC: (256326, 11)
df_reduce_RFC_instances: (886, 11)
df_reduce_RFC_instances hard: (886, 11)
df_reduce_RFC_instances_GLVQ: (2, 11)


## Función para entrenar y evaluar Voting Classifier

Se ha implementado un Voting Classifier combinando tres modelos: Gaussian Naive Bayes (GaussianNB), Support Vector Machine (SVM) y Random Forest, debido a que tienen fortalezas complementarias y diversidad de enfoques. GaussianNB es un modelo probabilístico que funciona bien con datos que tienen relaciones lineales simples. Por otro lado, SVM es ideal para encontrar fronteras de decisión óptimas, incluso en espacios de alta dimensionalidad o con relaciones no lineales entre características, especialmente cuando se utilizan kernels adecuados. Finalmente, Random Forest aporta robustez y capacidad de generalización al combinar múltiples árboles de decisión y manejar interacciones complejas entre características. Esta combinación permite que el Voting Classifier aproveche las fortalezas individuales de cada modelo, mitigando sus debilidades y asegurando un desempeño más equilibrado y preciso en una amplia variedad de problemas.

In [62]:
def train_and_evaluate_voting(X, y, test_data, columns_to_keep, hard=False):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Dimensiones de los conjuntos:")
    print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
    print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

    clf1 = GaussianNB()
    clf2 = RandomForestClassifier(random_state=42)
    clf3 = SVC(probability=True, random_state=42)

    if hard:
        voting_clf = VotingClassifier(estimators=[('nb', clf1), ('rf', clf2), ('svc', clf3)], voting='hard')
    else:
        voting_clf = VotingClassifier(estimators=[('nb', clf1), ('rf', clf2), ('svc', clf3)], voting='soft')

    param_grid = {
        'rf__n_estimators': [50, 100, 200],
        'rf__max_depth': [5, 10, 20, None],
        'svc__C': [0.1, 1, 10],
        'svc__kernel': ['linear', 'rbf'],
    }
    grid_search = GridSearchCV(estimator=voting_clf, param_grid=param_grid, cv=5, verbose=1, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    print("Mejores parámetros encontrados: ", grid_search.best_params_)

    best_model = grid_search.best_estimator_
    accuracy = best_model.score(X_test, y_test)
    print(f"Precisión en el conjunto de entrenamiento: {accuracy:.2f}")

    X_test_final = test_data[columns_to_keep]
    y_test_final = test_data['Class']

    scaler = MinMaxScaler()
    for col in ['Amount', 'Time']:
        if col in X_test_final.columns:
            X_test_final[col] = scaler.fit_transform(X_test_final[[col]])

    y_pred = best_model.predict(X_test_final)

    conf_matrix = confusion_matrix(y_test_final, y_pred)
    report = classification_report(y_test_final, y_pred, target_names=['Correctas', 'Fraudulentas'])

    print("Matriz de confusión:")
    print(conf_matrix)
    print("\nReporte de Clasificación:")
    print(report)

## Ejemplo con mRMR (ClusterCentroids_soft). Voting = soft

In [60]:
X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']
columns_to_keep_mrmr = ['V17', 'Time', 'Amount', 'V25', 'V20', 'V7', 'V13', 'V22', 'V19', 'V23']

print("\n--- Evaluación con mRMR ClusterCentroids_soft (Voting con modelos simples) ---")
train_and_evaluate_voting(X, y, test_data, columns_to_keep_mrmr)


--- Evaluación con mRMR ClusterCentroids_soft (Voting con modelos simples) ---
Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Mejores parámetros encontrados:  {'rf__max_depth': 10, 'rf__n_estimators': 50, 'svc__C': 0.1, 'svc__kernel': 'linear'}
Precisión en el conjunto de entrenamiento: 0.94


C:\Users\Usuario\AppData\Local\Temp\ipykernel_23948\740924741.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])
C:\Users\Usuario\AppData\Local\Temp\ipykernel_23948\740924741.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])


Matriz de confusión:
[[13249 15183]
 [    2    47]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.47      0.64     28432
Fraudulentas       0.00      0.96      0.01        49

    accuracy                           0.47     28481
   macro avg       0.50      0.71      0.32     28481
weighted avg       1.00      0.47      0.63     28481



En los resultados se observa como el modelo entrenado con estos datos no es capaz de hacer clasificaciones correctas, con una precisión de 0.00 y un f1-score de 0.01

## Ejemplo con mRMR (ClusterCentroids_soft). Voting = hard

In [63]:
X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']
columns_to_keep_mrmr = ['V17', 'Time', 'Amount', 'V25', 'V20', 'V7', 'V13', 'V22', 'V19', 'V23']

print("\n--- Evaluación con mRMR ClusterCentroids_soft (Voting hard con modelos simples) ---")
train_and_evaluate_voting(X, y, test_data, columns_to_keep_mrmr, hard=True)


--- Evaluación con mRMR ClusterCentroids_soft (Voting con modelos simples) ---
Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Mejores parámetros encontrados:  {'rf__max_depth': 10, 'rf__n_estimators': 50, 'svc__C': 10, 'svc__kernel': 'rbf'}
Precisión en el conjunto de entrenamiento: 0.94


C:\Users\Usuario\AppData\Local\Temp\ipykernel_23948\2074022474.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])
C:\Users\Usuario\AppData\Local\Temp\ipykernel_23948\2074022474.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])


Matriz de confusión:
[[16102 12330]
 [    3    46]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.57      0.72     28432
Fraudulentas       0.00      0.94      0.01        49

    accuracy                           0.57     28481
   macro avg       0.50      0.75      0.37     28481
weighted avg       1.00      0.57      0.72     28481



Se observa que usar el método hard de Voting no mejora el resultado.

## Ejemplo con mRMR (ClusterCentroids_hard). Voting = soft

In [36]:
X = df_reduce_mrmr_instances_hard.drop(columns=['Class'])
y = df_reduce_mrmr_instances_hard['Class']
columns_to_keep_mrmr = ['V17', 'Time', 'Amount', 'V25', 'V20', 'V7', 'V13', 'V22', 'V19', 'V23']

print("\n--- Evaluación con mRMR ClusterCentroids_hard (Voting con modelos simples) ---")
train_and_evaluate_voting(X, y, test_data, columns_to_keep_mrmr)


--- Evaluación con mRMR ClusterCentroids_hard (Voting con modelos simples) ---
Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Mejores parámetros encontrados:  {'knn__n_neighbors': 5, 'svc__C': 10, 'svc__kernel': 'rbf'}
Precisión en el conjunto de entrenamiento: 0.90


C:\Users\Usuario\AppData\Local\Temp\ipykernel_23948\487571183.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])
C:\Users\Usuario\AppData\Local\Temp\ipykernel_23948\487571183.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])


Matriz de confusión:
[[27463   969]
 [    5    44]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.97      0.98     28432
Fraudulentas       0.04      0.90      0.08        49

    accuracy                           0.97     28481
   macro avg       0.52      0.93      0.53     28481
weighted avg       1.00      0.97      0.98     28481



Usando vote hard en la reducción de instancias con cluster centroids, los resultados mejoran ligeramente con un 0.04 de precisión en la clase fraudulenta, y un 0.08 en el f1-score.

## Ejemplo con mRMR (ClusterCentroids_hard). Voting = hard

In [70]:
X = df_reduce_mrmr_instances_hard.drop(columns=['Class'])
y = df_reduce_mrmr_instances_hard['Class']
columns_to_keep_mrmr = ['V17', 'Time', 'Amount', 'V25', 'V20', 'V7', 'V13', 'V22', 'V19', 'V23']

print("\n--- Evaluación con mRMR ClusterCentroids_hard (Voting con modelos simples) ---")
train_and_evaluate_voting(X, y, test_data, columns_to_keep_mrmr, hard=True)


--- Evaluación con mRMR ClusterCentroids_hard (Voting con modelos simples) ---
Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Mejores parámetros encontrados:  {'rf__max_depth': 20, 'rf__n_estimators': 200, 'svc__C': 0.1, 'svc__kernel': 'linear'}
Precisión en el conjunto de entrenamiento: 0.91


C:\Users\Usuario\AppData\Local\Temp\ipykernel_23948\2074022474.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])
C:\Users\Usuario\AppData\Local\Temp\ipykernel_23948\2074022474.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])


Matriz de confusión:
[[27704   728]
 [    6    43]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.97      0.99     28432
Fraudulentas       0.06      0.88      0.10        49

    accuracy                           0.97     28481
   macro avg       0.53      0.93      0.55     28481
weighted avg       1.00      0.97      0.99     28481



El resultado experimenta una ligera mejora más si se usa el método voting=hard (precision en clase fraudulenta = 0.06 y f1-score en clase fraudulente = 0.10)

## Ejemplo con RFC (ClusterCentroids_soft). Voting = soft.

In [65]:
X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']
columns_to_keep_RFC = ['V17', 'V16', 'V12', 'V14', 'V11', 'V10', 'V9', 'V4', 'V18', 'V7']

print("\n--- Evaluación con RFC ClusterCentroids_soft (Voting Ensemble) ---")
train_and_evaluate_voting(X, y, test_data, columns_to_keep_RFC)


--- Evaluación con RFC ClusterCentroids_soft (Voting Ensemble) ---
Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Mejores parámetros encontrados:  {'rf__max_depth': 20, 'rf__n_estimators': 50, 'svc__C': 0.1, 'svc__kernel': 'linear'}
Precisión en el conjunto de entrenamiento: 0.90
Matriz de confusión:
[[28394    38]
 [    6    43]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      1.00      1.00     28432
Fraudulentas       0.53      0.88      0.66        49

    accuracy                           1.00     28481
   macro avg       0.77      0.94      0.83     28481
weighted avg       1.00      1.00      1.00     28481



Vemos como usar RFC para la reducción de características hace que el resultado mejore sustancialmente.

## Ejemplo con RFC (ClusterCentroids_soft). Voting = Hard

In [64]:
X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']
columns_to_keep_RFC = ['V17', 'V16', 'V12', 'V14', 'V11', 'V10', 'V9', 'V4', 'V18', 'V7']

print("\n--- Evaluación con RFC ClusterCentroids_soft (Voting Ensemble) ---")
train_and_evaluate_voting(X, y, test_data, columns_to_keep_RFC, hard=True)


--- Evaluación con RFC ClusterCentroids_soft (Voting Ensemble) ---
Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Mejores parámetros encontrados:  {'rf__max_depth': 10, 'rf__n_estimators': 50, 'svc__C': 10, 'svc__kernel': 'rbf'}
Precisión en el conjunto de entrenamiento: 0.92
Matriz de confusión:
[[28296   136]
 [    5    44]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      1.00      1.00     28432
Fraudulentas       0.24      0.90      0.38        49

    accuracy                           1.00     28481
   macro avg       0.62      0.95      0.69     28481
weighted avg       1.00      1.00      1.00     28481



Al cambiar Voting a hard en el clasificador, los resultados obtenidos empeoran pasando de una precisión en la clase fraudulenta de 0.53 a 0.24.

## Ejemplo con RFC (ClusterCentroids_hard). Voting = soft

In [37]:
X = df_reduce_RFC_instances_hard.drop(columns=['Class'])
y = df_reduce_RFC_instances_hard['Class']
columns_to_keep_RFC = ['V17', 'V16', 'V12', 'V14', 'V11', 'V10', 'V9', 'V4', 'V18', 'V7']

print("\n--- Evaluación con RFC ClusterCentroids_hard (Voting Ensemble) ---")
train_and_evaluate_voting(X, y, test_data, columns_to_keep_RFC)


--- Evaluación con RFC ClusterCentroids_soft (Voting Ensemble) ---
Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Mejores parámetros encontrados:  {'knn__n_neighbors': 5, 'svc__C': 0.1, 'svc__kernel': 'linear'}
Precisión en el conjunto de entrenamiento: 0.91
Matriz de confusión:
[[28396    36]
 [    6    43]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      1.00      1.00     28432
Fraudulentas       0.54      0.88      0.67        49

    accuracy                           1.00     28481
   macro avg       0.77      0.94      0.84     28481
weighted avg       1.00      1.00      1.00     28481



Usar el método hard en cluster centroids hace que la precision y el f1-score mejoren en un punto de 0.53 a 0.54, y de 0.66 a 0.67 respectivamente.

## Ejemplo con RFC (ClusterCentroids_hard). Voting = hard

In [71]:
X = df_reduce_RFC_instances_hard.drop(columns=['Class'])
y = df_reduce_RFC_instances_hard['Class']
columns_to_keep_RFC = ['V17', 'V16', 'V12', 'V14', 'V11', 'V10', 'V9', 'V4', 'V18', 'V7']

print("\n--- Evaluación con RFC ClusterCentroids_hard (Voting Ensemble) ---")
train_and_evaluate_voting(X, y, test_data, columns_to_keep_RFC, hard=True)


--- Evaluación con RFC ClusterCentroids_hard (Voting Ensemble) ---
Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Mejores parámetros encontrados:  {'rf__max_depth': 10, 'rf__n_estimators': 100, 'svc__C': 10, 'svc__kernel': 'rbf'}
Precisión en el conjunto de entrenamiento: 0.92
Matriz de confusión:
[[28303   129]
 [    5    44]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      1.00      1.00     28432
Fraudulentas       0.25      0.90      0.40        49

    accuracy                           1.00     28481
   macro avg       0.63      0.95      0.70     28481
weighted avg       1.00      1.00      1.00     28481



Una vez más, el rendimiento empeora cuando el método de voto en el clasificador es hard.

## Función para entrenar y evaluar Voting Classifier usando ensembles.

Se realiza una nueva función para entrenar un Voting Classifier que use tres ensembles en lugar de clasificadores más simples para tratar de obtener una mejora.

In [66]:
def train_and_evaluate_voting_ensemble(X, y, test_data, columns_to_keep):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Dimensiones de los conjuntos:")
    print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
    print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

    clf1 = RandomForestClassifier(random_state=42)
    clf2 = GradientBoostingClassifier(random_state=42)
    clf3 = AdaBoostClassifier(random_state=42)

    voting_clf = VotingClassifier(estimators=[('rf', clf1), ('gb', clf2), ('ada', clf3)], voting='soft')

    param_grid = {
        'rf__n_estimators': [50, 100, 200],
        'rf__max_depth': [5, 10, 20, None],
        'gb__n_estimators': [50, 100, 200],
        'gb__learning_rate': [0.01, 0.05, 0.1, 0.2],
        'ada__n_estimators': [50, 100, 200],
    }
    grid_search = GridSearchCV(estimator=voting_clf, param_grid=param_grid, cv=5, verbose=1, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    print("Mejores parámetros encontrados: ", grid_search.best_params_)

    best_model = grid_search.best_estimator_
    accuracy = best_model.score(X_test, y_test)
    print(f"Precisión en el conjunto de entrenamiento: {accuracy:.2f}")

    X_test_final = test_data[columns_to_keep]
    y_test_final = test_data['Class']

    scaler = MinMaxScaler()
    for col in ['Amount', 'Time']:
        if col in X_test_final.columns:
            X_test_final[col] = scaler.fit_transform(X_test_final[[col]])

    y_pred = best_model.predict(X_test_final)

    conf_matrix = confusion_matrix(y_test_final, y_pred)
    report = classification_report(y_test_final, y_pred, target_names=['Correctas', 'Fraudulentas'])

    print("Matriz de confusión:")
    print(conf_matrix)
    print("\nReporte de Clasificación:")
    print(report)


## Ejemplo con mRMR (ClusterCentroids_soft)

In [72]:
X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']
columns_to_keep_mrmr = ['V17', 'Time', 'Amount', 'V25', 'V20', 'V7', 'V13', 'V22', 'V19', 'V23']

print("\n--- Evaluación con mRMR ClusterCentroids_soft (Voting con modelos simples) ---")
train_and_evaluate_voting_ensemble(X, y, test_data, columns_to_keep_mrmr)


--- Evaluación con mRMR ClusterCentroids_soft (Voting con modelos simples) ---
Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Mejores parámetros encontrados:  {'ada__n_estimators': 50, 'gb__learning_rate': 0.05, 'gb__n_estimators': 200, 'rf__max_depth': 10, 'rf__n_estimators': 50}
Precisión en el conjunto de entrenamiento: 0.94


C:\Users\Usuario\AppData\Local\Temp\ipykernel_23948\2562104302.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])
C:\Users\Usuario\AppData\Local\Temp\ipykernel_23948\2562104302.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])


Matriz de confusión:
[[16301 12131]
 [    2    47]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.57      0.73     28432
Fraudulentas       0.00      0.96      0.01        49

    accuracy                           0.57     28481
   macro avg       0.50      0.77      0.37     28481
weighted avg       1.00      0.57      0.73     28481



Observamos que con mrMr, el resultado sigue siendo muy malo.

## Ejemplo con RFC (ClusterCentroids_soft)

In [67]:
X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']
columns_to_keep_RFC = ['V17', 'V16', 'V12', 'V14', 'V11', 'V10', 'V9', 'V4', 'V18', 'V7']

print("\n--- Evaluación con RFC ClusterCentroids_soft (Voting Ensemble) ---")
train_and_evaluate_voting_ensemble(X, y, test_data, columns_to_keep_RFC)


--- Evaluación con RFC ClusterCentroids_soft (Voting Ensemble) ---
Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Mejores parámetros encontrados:  {'ada__n_estimators': 100, 'gb__learning_rate': 0.01, 'gb__n_estimators': 50, 'rf__max_depth': 10, 'rf__n_estimators': 200}
Precisión en el conjunto de entrenamiento: 0.92
Matriz de confusión:
[[27983   449]
 [    5    44]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.98      0.99     28432
Fraudulentas       0.09      0.90      0.16        49

    accuracy                           0.98     28481
   macro avg       0.54      0.94      0.58     28481
weighted avg       1.00      0.98      0.99     28481



Al observar que con RFC los resultados son tan inferiores a los obtenidos con el anterior modelo, se detiene la experimentación con este nuevo clasificador.

## Conclusiones
Voting=Soft
| Modelo                   | Precisión train | Precisión test | f1 test |
|--------------------------|-----------------|----------------|---------|
| mrMr - CC soft           | 0.94           | 0.00           | 0.01    |
| mrMr - CC hard           | 0.90              | 0.04              | 0.08       |
| RFC - CC soft            | 0.90           | 0.53           | 0.66    |
| RFC - CC hard            | 0.91              | 0.54              | 0.67       |
| mrMr - CC soft (ensembles) | 0.94         | 0.00           | 0.01    |
| RFC - CC soft (ensembles) | 0.92             | 0.09              | 0.16       |

Voting=Hard
| Modelo          | Precisión train | Precisión test | f1 test |
|------------------|-----------------|----------------|---------|
| mrMr - CC soft   | 0.94           | 0.00           | 0.01    |
| mrMr - CC hard   | 0.91              | 0.06              | 0.10       |
| RFC - CC soft    | 0.92           | 0.24           | 0.38    |
| RFC - CC hard    | 0.92              | 0.25              | 0.40       |


En ambas tablas, los modelos basados en mrMr - CC soft muestran un alto desempeño en el entrenamiento (Precisión train = 0.94) pero un desempeño extremadamente bajo en las pruebas (Precisión test = 0.00 y f1 test ≈ 0.01). Este comportamiento indica que los modelos mrMr tienen un fuerte sobreajuste, ya que no logran generalizar los resultados en los datos de prueba.

Los modelos RFC - CC soft tienen una precisión de entrenamiento menor en comparación con mrMr (0.90 en la primera tabla y 0.92 en la segunda). Sin embargo, logran resultados significativamente mejores en las pruebas:
- Tabla 1: Precisión test = 0.53 y f1 test = 0.66.
- Tabla 2: Precisión test = 0.24 y f1 test = 0.38.
Esto sugiere que los modelos RFC tienen mejor capacidad de generalización que mrMr.

En la Tabla 1, el desempeño del ensamble con mrMr no mejora respecto al modelo base (Precisión test = 0.00, f1 test ≈ 0.01). Esto indica que el método de ensamble no logró solucionar el problema de sobreajuste. Además, la baja generalización alcanzada también con RFC indica que el método de ensamble aplicado aquí no logra capturar correctamente las características de los datos de prueba.

- Los modelos mrMr tienen un desempeño notablemente pobre en generalización y requieren ajustes para evitar el sobreajuste.
- Los modelos RFC, aunque no perfectos, presentan mejores métricas en el conjunto de pruebas.

Con voting=hard, los resultados en pruebas tienden a ser más bajos para RFC. Esto podría deberse a que el hard voting introduce más ruido al basarse en decisiones discretas en lugar de promedios ponderados de probabilidad.